Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train_v2.csv')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/Users/antelinvestigacionydesarrollo/anaconda3/envs/cursods/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [4]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [5]:
all_data.target.mean()

0.3342730567123426

In [6]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


#### Method 2

In [7]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621791


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [8]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False, random_state=2019)

new_data = list()
for idx_train, idx_val in kf.split(all_data):
    train_data = all_data.iloc[idx_train]
    val_data = all_data.iloc[idx_val].copy()
    item_id_target_mean = train_data.groupby('item_id').target.mean()
    val_data['item_target_enc'] = val_data['item_id'].map(item_id_target_mean)
    new_data.append(val_data)

all_data = pd.concat(new_data)

# Fill NaNs
all_data['item_target_enc'].fillna(all_data.target.mean(), inplace=True)

encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.41645904885340546
Current answer for task KFold_scheme is: 0.41645904885340546


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [10]:
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform(
    lambda x: (x.sum() - x) / (x.shape[0] - 1)
)
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129305
Current answer for task Leave-one-out_scheme is: 0.480384831129305


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [11]:
alpha = 100
global_mean = 0.3343
n_rows = all_data.groupby('item_id').target.count()
means = all_data.groupby('item_id').target.mean()

smooth_means = (means * n_rows + alpha * global_mean) / (n_rows + alpha)

all_data['item_target_enc'] = all_data.item_id.map(smooth_means)
encoded_feature = all_data['item_target_enc']

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.4818198797097282
Current answer for task Smoothing_scheme is: 0.4818198797097282


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [102]:
all_data.drop('target_enc_item', axis=1, inplace=True)
#all_data.drop('target_enc_item_y', axis=1, inplace=True)
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
0,0,19,0,0.0,0.237448
1,0,27,0,0.0,0.089905
2,0,28,0,0.0,0.168964
3,0,29,0,0.0,0.107910
4,0,32,0,6.0,1.260635


First, let's get a new df that aggregates the item's targets per month.

In [103]:
months = all_data.date_block_num.unique()

In [104]:
item_ids = all_data.item_id.unique()

In [105]:
data_agg = all_data[['item_id', 'date_block_num', 'target']]
agg_targets = data_agg.groupby(['item_id', 'date_block_num']).sum().reset_index()
agg_targets = agg_targets.sort_values(by=['item_id', 'date_block_num'])

In [106]:
agg_targets.head(10)

,item_id,date_block_num,target
0,0,20,1.0
1,1,15,2.0
2,1,18,1.0
3,1,19,1.0
4,1,20,1.0
5,1,21,1.0
6,2,19,1.0
7,2,22,1.0
8,3,18,1.0
9,3,19,1.0


In [107]:
from itertools import product

In [108]:
data_list = list()
for item, month in product(item_ids, months):
    data_list.append({'item_id': item, 'date_block_num': month})

In [109]:
all_months = pd.DataFrame(data_list).sort_values(by=['item_id', 'date_block_num'])

In [110]:
all_months.head()

,date_block_num,item_id
585718,0,0
585719,1,0
585720,2,0
585721,3,0
585722,4,0


In [111]:
all_months = all_months.merge(agg_targets, on=['item_id', 'date_block_num'], how='left')
all_months.fillna(0, inplace=True)

In [112]:
all_months.head(40)

,date_block_num,item_id,target
0,0,0,0.0
1,1,0,0.0
2,2,0,0.0
3,3,0,0.0
4,4,0,0.0
5,5,0,0.0
6,6,0,0.0
7,7,0,0.0
8,8,0,0.0
9,9,0,0.0


In [113]:
global_mean = 0.3343

all_months['cumsum'] = all_months.groupby('item_id')['target'].cumsum()
all_months['cumcnt'] = all_months.groupby('item_id')['target'].cumcount() + 1
all_months['target_enc_item'] = all_months['cumsum'] / all_months['cumcnt']
all_months.fillna(global_mean, inplace=True)

In [114]:
all_months.head(40)

,date_block_num,item_id,target,cumsum,cumcnt,target_enc_item
0,0,0,0.0,0.0,1,0.000000
1,1,0,0.0,0.0,2,0.000000
2,2,0,0.0,0.0,3,0.000000
3,3,0,0.0,0.0,4,0.000000
4,4,0,0.0,0.0,5,0.000000
5,5,0,0.0,0.0,6,0.000000
6,6,0,0.0,0.0,7,0.000000
7,7,0,0.0,0.0,8,0.000000
8,8,0,0.0,0.0,9,0.000000
9,9,0,0.0,0.0,10,0.000000


In [115]:
all_months.target_enc_item.mean()

5.568818754644051

In [116]:
all_data = all_data.merge(all_months[['item_id', 'date_block_num', 'target_enc_item']],
                         on=['item_id', 'date_block_num'], how='left')
all_data.head(10)

,shop_id,item_id,date_block_num,target,item_target_enc,target_enc_item
0,0,19,0,0.0,0.237448,1.0
1,0,27,0,0.0,0.089905,7.0
2,0,28,0,0.0,0.168964,8.0
3,0,29,0,0.0,0.107910,4.0
4,0,32,0,6.0,1.260635,299.0
5,0,33,0,3.0,0.515676,61.0
6,0,34,0,0.0,0.166235,9.0
7,0,35,0,1.0,0.762478,78.0
8,0,40,0,0.0,0.089425,4.0
9,0,41,0,0.0,0.127171,9.0


In [117]:
all_data.tail(10)

,shop_id,item_id,date_block_num,target,item_target_enc,target_enc_item
10913840,59,22118,33,0.0,0.257813,3.558824
10913841,59,22139,33,0.0,0.228013,10.323529
10913842,59,22145,33,0.0,0.597010,9.411765
10913843,59,22154,33,0.0,0.145102,1.735294
10913844,59,22158,33,0.0,0.239097,0.029412
10913845,59,22162,33,0.0,1.334117,20.558824
10913846,59,22163,33,0.0,0.473493,2.205882
10913847,59,22164,33,0.0,1.054970,14.500000
10913848,59,22166,33,0.0,0.298926,10.235294
10913849,59,22167,33,0.0,1.024550,38.823529


### Let's try the easy way

In [121]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,target_enc_item
0,0,19,0,0.0,0.237448,1.0
1,0,27,0,0.0,0.089905,7.0
2,0,28,0,0.0,0.168964,8.0
3,0,29,0,0.0,0.107910,4.0
4,0,32,0,6.0,1.260635,299.0


In [122]:
cumsum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']
cumcnt = all_data.groupby('item_id')['target'].cumcount()
all_data['simple_target_enc'] = cumsum / cumcnt
all_data['simple_target_enc'].fillna(global_mean, inplace=True)
all_data.head(15)

,shop_id,item_id,date_block_num,target,item_target_enc,target_enc_item,simple_target_enc
0,0,19,0,0.0,0.237448,1.0,0.3343
1,0,27,0,0.0,0.089905,7.0,0.3343
2,0,28,0,0.0,0.168964,8.0,0.3343
3,0,29,0,0.0,0.107910,4.0,0.3343
4,0,32,0,6.0,1.260635,299.0,0.3343
5,0,33,0,3.0,0.515676,61.0,0.3343
6,0,34,0,0.0,0.166235,9.0,0.3343
7,0,35,0,1.0,0.762478,78.0,0.3343
8,0,40,0,0.0,0.089425,4.0,0.3343
9,0,41,0,0.0,0.127171,9.0,0.3343


In [123]:
all_data.tail(15)

,shop_id,item_id,date_block_num,target,item_target_enc,target_enc_item,simple_target_enc
10913835,59,22102,33,1.0,4.850780,50.235294,6.616279
10913836,59,22104,33,0.0,0.759801,9.117647,0.883191
10913837,59,22105,33,0.0,0.116862,3.058824,0.096744
10913838,59,22106,33,0.0,0.144872,3.852941,0.126692
10913839,59,22111,33,0.0,0.130741,5.500000,0.117981
10913840,59,22118,33,0.0,0.257813,3.558824,0.242972
10913841,59,22139,33,0.0,0.228013,10.323529,0.221451
10913842,59,22145,33,0.0,0.597010,9.411765,0.651731
10913843,59,22154,33,0.0,0.145102,1.735294,0.110075
10913844,59,22158,33,0.0,0.239097,0.029412,0.023256


In [124]:
encoded_feature = all_data['simple_target_enc']

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211081701
Current answer for task Expanding_mean_scheme is: 0.5025245211081701


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [125]:
STUDENT_EMAIL = 'miguel.tasende@gmail.com'
STUDENT_TOKEN =
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645904885340546
Task Leave-one-out_scheme: 0.480384831129305
Task Smoothing_scheme: 0.4818198797097282
Task Expanding_mean_scheme: 0.5025245211081701


In [126]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
